*Sometimes the rabbit hole goes deeper than you thought.*

[![](./01.png)](https://steemit.com/steem/@lextenebris/steem-the-whale-wars-bernie-and-haejin-sitting-in-a-tree-structure#@crokkon/re-lextenebris-re-crokkon-re-lextenebris-re-abh12345-re-lextenebris-re-abh12345-re-lextenebris-steem-the-whale-wars-bernie-and-haejin-sitting-in-a-tree-structure-20180307t161114284z)

And so it began, with a simple back and forth between @crokkon and I, talking about the methods of deriving `steem-per-mvest` and whether or not it was even worth doing.

This is going to be an example of how asking the simplest questions can take you down some very strange rabbit holes, and if you're willing to follow them you can stumble on things you have no way of understanding. But at least you found them.

The basic question:

Has the effective ratio between steem and mega-vests changed significantly enough over the last year to even bother worrying about generating a precise ratio in order to determine how much a given vote is worth?

It seems like a very simple question to answer. And it is! But digging out the answer ends up revealing more than it intends.

## Getting Out the Shovel

Like most things lately that involve code, this starts simply enough by loading up the basics of SteamData, pulling in some useful analysis tools, and finishing off the imports with something to draw pretty pictures. I know up front that I'm going to want to plot at least one graph of the ratio between deposits and withdrawals in the table of operations of the system fulfilling requests to convert steem to SP.

If you've been following my code posts over the last few weeks, you know what's coming up now. Imports, database initialization, query, and then we start looking at the data we get back.

In [1]:
# Setting up the imports 

from steemdata import SteemData
import datetime
from datetime import datetime as dt

import numpy as np

import bokeh
import bokeh.plotting as bplt

In [2]:
# Init connection to database

db = SteemData()

In [3]:
query = {
    'type' : 'fill_vesting_withdraw',
    'timestamp' : {'$gte': dt.now() - datetime.timedelta(days=360)}}
    
proj = {'deposited.amount': 1, 'withdrawn.amount': 1, 'timestamp': 1, 'from_account': 1, 'to_account':1, '_id': 0}

sort = [('timestamp', -1)]

In [4]:
%%time

result = db.Operations.find(query,
                            projection=proj,
                            sort=sort)

fvL = list(result)

Wall time: 1min 33s


I decided to go big rather than to go home and just grab the last year of transaction information on vesting withdrawals. That sounds like complete overkill, and it probably is, but as a result I can look at the data and really get a feeling for what's going on under the hood. Also, it gives us a chance to make a solid prediction (technically hindcast) of what the ratio was and how much it's varied.

It turns out that there are just over a million vesting withdrawals that occurred in the last year. Honestly, that could be a lot worse. We can really work with this amount of data.

While were looking, let's check out the first five and the last five in the list. Since I told the database to return the list in reverse chronological order, those at the top of the most recent and of those at the bottom are the oldest.

In [5]:
len(fvL)

1049438

In [6]:
fvL[:5], fvL[-5:]

([{'deposited': {'amount': 35.77},
   'from_account': 'zakariashikder',
   'timestamp': datetime.datetime(2018, 3, 8, 18, 18, 33),
   'to_account': 'zakariashikder',
   'withdrawn': {'amount': 73050.513285}},
  {'deposited': {'amount': 20.336},
   'from_account': 'nobutsd',
   'timestamp': datetime.datetime(2018, 3, 8, 18, 17, 33),
   'to_account': 'nobutsd',
   'withdrawn': {'amount': 41531.82173}},
  {'deposited': {'amount': 0.927},
   'from_account': 'junhokim',
   'timestamp': datetime.datetime(2018, 3, 8, 18, 17, 12),
   'to_account': 'junhokim',
   'withdrawn': {'amount': 1894.72076}},
  {'deposited': {'amount': 10.278},
   'from_account': 'jakiasultana',
   'timestamp': datetime.datetime(2018, 3, 8, 18, 15, 45),
   'to_account': 'jakiasultana',
   'withdrawn': {'amount': 20991.217037}},
  {'deposited': {'amount': 2.673},
   'from_account': 'markboss',
   'timestamp': datetime.datetime(2018, 3, 8, 18, 15, 33),
   'to_account': 'markboss',
   'withdrawn': {'amount': 5459.346342}}]

There's nothing particularly shocking or surprising about this information really.

Well, that and the fact that @tinfoilfedora, who showed up once, has one of the best names that I've ever seen on the platform. Bravo!

Now that we have the raw data that we came for, we need to come up with the actual ratio in question. This is relatively easy if we just make a list comprehension which pulls out the necessary values and gives them a quick divide. The ratio just falls out.

While we're at it, let's look at the first and last five just to see if the data looks relatively coherent and there's nothing obviously wrong with it.

In [7]:
spmL = [e['deposited']['amount'] / e['withdrawn']['amount']
        for e in fvL]

In [8]:
spmL[:5], spmL[-5:]

([0.0004896611726798767,
  0.0004896486393543035,
  0.0004892541526805249,
  0.000489633353887179,
  0.000489619055570078],
 [0.0004806019912260807,
  0.0004800960347037997,
  0.0004806003473206151,
  0.00048059427802293165,
  0.00048055643711602])

In [9]:
np.median(spmL)

0.00048264105814548454

Things look pretty good in terms of raw data. Not only that, but we can tell from just an easy eyeball of the information that steem per million vests hasn't really changed all that much in the last year. The only really conspicuous changes down in the millionths place, which might be meaningful to the really big whales in the pool but for most of the people reading this – that's probably less than noise in the signal.

Just because we have the information, I decided to pull the median out. Not the average, not the mean, not even the mode – the median. For those not familiar with statistical operations, that's what you get when you sort a list of numbers highest to lowest and literally pick the one that is in the middle of the list.

This data is not entirely clean, as you will see later, but because the sample is so large, the median was going to pull out the most reasonable value in the structure. It passes the sniff test as a reasonable value in between those at the beginning and and of the last year.

## Table That Motion

@crokkon was good enough to share [a link to some code that he had written a little over two weeks ago which did a little dumpster diving in the database to do these calculations and work out the ratio over time.](https://steemit.com/steemdev/@crokkon/re-schererf-re-crokkon-re-schererf-tutorial-get-the-value-of-your-steemit-earnings-part-2-calculate-steem-power-20180219t153715756z) He cleverly managed to avoid showing off the filters that he had implemented "for simplicity", but I figured it couldn't be that big a deal.

It's actually a pretty big deal. We'll take a look at why here shortly.

First, though, let's shove some data into a structure which is designed for manipulating tabular data.

[Pandas.](http://pandas.pydata.org/pandas-docs/stable/10min.html)

I know that a lot of coders on the platform are already familiar with pandas because it is one of the best known big data manipulating tools in the world. It's new to me. That probably doesn't say a lot positive about my experience as an analyst, but bear with me – we'll run with it.

After all, you've made it this far.

To shove the data from the database into a form that we can manipulate easily, we'll basically just implement a series of implicit for loops in the form of some list comprehensions. Essentially we just build a nice record dictionary in a semi-lazy way, fold it together with the list of ratios that we generated, and incidentally tell the data frame that the bit of information that primarily differentiates one of these entries from the other is the timestamp.

That last bit is important because pandas has some nice tools for working with timeseries, which we'll talk about shortly.

In [10]:
import pandas as pd

In [11]:
Data = pd.DataFrame({'deposited': [e['deposited']['amount'] for e in fvL],
                     'withdrawn': [e['deposited']['amount'] for e in fvL],
                     'from_account': [e['from_account'] for e in fvL],
                     'to_account': [e['to_account'] for e in fvL],
                     'ratio': spmL
                    }, 
                     index=[e['timestamp'] for e in fvL])

Now we have a nice, flexible, high-speed data structure – which is full of data with questionable consistency.

Having spent a little bit of time trawling through this pile, there were two things that immediately leapt out at me as indicators of less than useful knowledge. Firstly, some of the calculated ratios were zero. That can really only occur when the deposit is equal to zero. There were also sometimes when the ratio was 1, which can only happen when – well, nothing good.

So let's filter out bits of the database where the ratio is greater than 0.99 and entries where deposits are zero.

In [12]:
Data = Data[(Data['ratio'] < 0.99)]
Data = Data[(Data['deposited'] > 0)]

One of the nice things about pandas is that when given tabular data, it's really quite nice about creating a clear output.

Again, let's look at the top and the bottom of this list since it's still in chronological order. Now we have a nice temporal index, and all of our fields are nicely lined up.

Again, the ratios make sense and pandas is very good about giving us just enough information to see where there is some differentiation.

In [13]:
Data.head()

,deposited,from_account,ratio,to_account,withdrawn
2018-03-08 18:18:33,35.770,zakariashikder,0.000490,zakariashikder,35.770
2018-03-08 18:17:33,20.336,nobutsd,0.000490,nobutsd,20.336
2018-03-08 18:17:12,0.927,junhokim,0.000489,junhokim,0.927
2018-03-08 18:15:45,10.278,jakiasultana,0.000490,jakiasultana,10.278
2018-03-08 18:15:33,2.673,markboss,0.000490,markboss,2.673


In [14]:
Data.tail()

,deposited,from_account,ratio,to_account,withdrawn
2017-03-13 13:40:15,2337.791,salva82,0.000481,salva82,2337.791
2017-03-13 13:36:48,0.689,romangelsi,0.000480,romangelsi,0.689
2017-03-13 13:32:18,65.278,midnightoil,0.000481,midnightoil,65.278
2017-03-13 13:32:09,43.866,catulhu,0.000481,catulhu,43.866
2017-03-13 13:31:36,5.902,dennygalindo,0.000481,dennygalindo,5.902


Since we have everything in such a nice structure, we can query it directly for various manipulations rather than having to write the code to do all those things by hand.

For instance, what if we just want to pileup all the accounts that have received these withdrawals and add up the total amount deposited and withdrawn over the last year. If we were to actually write the Python code for that, it would involve at least one loop, a set to hold the accounts and unify them, and probably an accumulator.

With a panda data frame? It's a one-liner.

In [15]:
Data.groupby('to_account')['deposited', 'withdrawn'].sum()

,deposited,withdrawn
to_account,,
a-a,6.153,6.153
a-a-lifemix,216.746,216.746
a-angel,6.785,6.785
a-c-s,7.721,7.721
a-condor,737.694,737.694
a-eye,6.461,6.461
a-team,6.009,6.009
a0de6qzchbut,1836.710,1836.710
a1-jonesblogger,4.917,4.917


The interesting thing here is that of that 1 million set of implicit withdrawals, only 32,000 accounts actually received funds.

That's – kind of surprising. This is the kind of thing that perks my ears when stuff rolls around. Essentially, this is a much smaller number of accounts than I ever expected to see involved over the last year.

I can't even really begin to theorize about why these numbers are so small. I find it more than a little concerning. There are a lot of promotional people who constantly tout the number of active accounts which often exceed 70,000 – but here we see 32,000 accounts which have triggered this operation on the blockchain.

I can't explain that. Yet.

We will leave aside that mystery for the moment and make use of pandas again to do something that would be much harder without it – carving all of these transactions up into weekly blocks. Again, something that in straight Python would be a real challenge but here turns into a single line where we simply tell the structure to resample itself and break down into one-week chunks, taking the numeric values within those chunks and taking the median value.

Yes, I tried using the average value. Because of some of the weird bits going on under the hood, the mean was just a bad choice. The median is just fine for our purposes, and even know it's probably not the right thing for the deposited or withdrawn values, I think it's interesting to look at because one of the things that we can tell immediately is that the usual value is relatively low.

52 weeks ago, there was a particularly odd outlier in values. One day, that might be worth investigating more thoroughly. Everything after that looks pretty normal.

In [16]:
WkData = Data.resample('1W').median()

WkData

,deposited,ratio,withdrawn
2017-03-19,2.2810,0.000481,2.2810
2017-03-26,2.5440,0.000481,2.5440
2017-04-02,2.6260,0.000481,2.6260
2017-04-09,1.7650,0.000481,1.7650
2017-04-16,2.3390,0.000481,2.3390
2017-04-23,2.4710,0.000481,2.4710
2017-04-30,2.1810,0.000482,2.1810
2017-05-07,2.4730,0.000482,2.4730
2017-05-14,2.4740,0.000482,2.4740
2017-05-21,1.0300,0.000482,1.0300


Another thing that you learn while dumpster diving in data is to look for strange aberrations. Anything different is something worth looking at.

Strangest thing I've seen in this database? That some of these transactions don't have the same from account and to account. It's not even a particularly small portion of the database. Out of 1 million transactions, almost 300,000 have different origin and destination accounts.

No, I don't have a really good explanation for why that's the case, either. It's far too common to be unusual behavior, but there's something about the way that these withdrawal transactions are happening that I don't understand.

In [17]:
MultData = Data[(Data['from_account'] != Data['to_account'])]

In [18]:
MultData

,deposited,from_account,ratio,to_account,withdrawn
2018-03-08 17:08:33,0.397,jennykidd,0.000489,cabi5boh,0.397
2018-03-08 17:08:18,0.415,bitrexx,0.000489,cabi5boh,0.415
2018-03-08 15:45:09,0.387,index178,0.000489,sigmajin,0.387
2018-03-08 15:45:09,0.387,index180,0.000489,sigmajin,0.387
2018-03-08 15:45:09,0.387,index181,0.000489,sigmajin,0.387
2018-03-08 15:45:09,0.387,index182,0.000489,sigmajin,0.387
2018-03-08 15:45:09,0.387,index183,0.000489,sigmajin,0.387
2018-03-08 15:45:09,0.387,index184,0.000489,sigmajin,0.387
2018-03-08 15:45:09,0.387,index185,0.000489,sigmajin,0.387
2018-03-08 15:45:09,0.387,index186,0.000489,sigmajin,0.387


While we've got the information out, let's see what happens when we group by the receiving account.

In [19]:
MultData.groupby('to_account').mean()

,deposited,ratio,withdrawn
to_account,,,
aaron,2621.641637,0.000487,2621.641637
adiel,3.751456,0.000483,3.751456
ahmeddimassi,0.836500,0.000489,0.836500
aizensou,7.521411,0.000481,7.521411
alittle,21509.356333,0.000486,21509.356333
alot,72956.143818,0.000486,72956.143818
alpha,7272.757897,0.000481,7272.757897
anastacia,81067.693000,0.000483,81067.693000
another,2.276687,0.000481,2.276687


Now, this is where it gets strange. There are only 187 distinct recipient accounts for these 300,000 withdrawal transactions.

It was looking a little strange, before – but anytime you see a sample this large have less than 200 of anything happening, it's worth looking at.

I went with the mean derivation on this one because it seemed like an actual average for each of these values would be meaningful. They are certainly interesting. Some of these things are surprisingly vast, while in general most of the leakage effects are relatively small.

Again, I have no idea what's going on here – but because there are only 187 distinct rows that's a series of relationships which we can easily graph, and we will – after we do a few other things first.

## Do You Think My Slope Is Pretty?

It's time, finally, to answer the question we started this whole thing with. We already have a pretty good idea, but a picture is worth a thousand words. Which probably means that a graph is worth about half of what I've written already about this problem?

When I put it that way, it starts to be kind of sad.

Rather than use [plotly](https://plot.ly/) today, I decided to learn an entirely new graphing system, because if you don't learn at least three new complicated things a day what good are you? Besides, while plotly does make some really nice online graphs – they really do require that a server elsewhere crunch the data, and I am almost certain to have more hardware sitting here under me to do this graph crunching than any online services likely to let me have for free.

I was right about that. Crunching the graphs with [bokeh](https://bokeh.pydata.org/en/latest/) took almost 0 time and generated a very nice HTML interface locally which could be saved out to a convenient image.

Setting up a graph is a little bit fiddly and I don't have exactly what I wanted – but it's good enough for government work which means it's more than good enough for this job.

In [20]:
bplt.output_file('weeklyRatio.html')

In [21]:
Plot = bplt.figure( tools='pan,box_zoom,reset,save',
                              title='Median Ratio Per Week For a Year',
                              x_axis_label='Week', y_axis_label='Ratio',
                              x_axis_type='datetime',
                              width=600, height=400, sizing_mode='scale_height'
                            )

Plot.axis.minor_tick_in = -3
Plot.axis.minor_tick_out = 6

Plot.yaxis[0].formatter = bokeh.models.NumeralTickFormatter(format="0.000000")

# Plot.yaxis.major_label_orientation = "vertical"

Plot.grid.grid_line_alpha = 0.4
Plot.ygrid.minor_grid_line_color = 'grey'
Plot.ygrid.minor_grid_line_alpha = 0.2

Plot.ygrid.band_fill_color='olive'
Plot.ygrid.band_fill_alpha=0.1

Plot.line(WkData.index, WkData.ratio, color='red', legend='Ratio')

GlyphRenderer(id='3222d0e2-71d7-4941-b19c-fb8a2beecefa', ...)

In [22]:
bplt.show(Plot)

![](./00.png)

This thing looks like a climate change graph, but it has something in common with global warming.

The slope of that curve is nowhere near as intense as it appears on the sheet.

Notice what the range on the Y axis actually is. It just runs over the range that we've already seen, differing by 1/100000 from the top to the bottom. I also used the weekly numbers rather than the full 1 million element set because there was absolutely no need for that level of resolution. 52 points along the timeline are more than sufficient.

While the slope isn't particularly useful for us to know, the fact that it exists is interesting. There has been a general trend upwards on the steem per mega-vests over the last year. As I understand it, that could only be the result of witnesses making a conscious decision to increase that number ever so slightly.

Again, I could be completely wrong about that. Once you start getting down to this level of poking at the blockchain, there is a lot of mystery and a lot of absolute absence of documentation, with the only reference that you can look at which makes a difference being the source code.

I haven't quite descended to that level of madness. Yet.

## Who's Touching Who?

Let's take the pause that refreshes and look back up at those accounts which make up 30% of the overall transactions of this nature but only target 178 accounts or so. What's going on there?

I have no idea what's going on there, but we can see what that network of accounts actually looks like. All we need to do is create a relationship map which sums the transfers between them and look at what's in front of us.

That's going to take our old friend [graphviz](https://www.graphviz.org/), so let's get that set up.

In [42]:
from graphviz import Digraph

In [78]:
dot = Digraph(comment="Steem-to-Mvest Oddity Map", 
              format="svg",
              engine="sfdp")

In [79]:
dot.attr('graph', overlap='false')
dot.attr('graph', ratio='auto')
dot.attr('graph', size='1000000,1000000')
dot.attr('graph', start='1.0')
dot.attr('graph', K='10')
dot.attr('graph', margin='5')

We'll start with accounts being light grey, just for some visual texture.

In [80]:
dot.attr('node', shape='rectangle', 
         style='filled', color='black', 
         fillcolor='lightgrey')

Now that we have a framework, there needs to be some data. We'll build a new data frame to give us something to work with.

No timeline info is going to be necessary, which is nice. All that can go. Just accounts and values of what's been withdrawn.

In [81]:
rGraphData = pd.DataFrame({'to': MultData.to_account,
                           'from': MultData.from_account,
                           'deposit': MultData.deposited,
                           'withdraw': MultData.withdrawn})

In [82]:
rGraphData.head(20)

,deposit,from,to,withdraw
2018-03-08 17:08:33,0.397,jennykidd,cabi5boh,0.397
2018-03-08 17:08:18,0.415,bitrexx,cabi5boh,0.415
2018-03-08 15:45:09,0.387,index178,sigmajin,0.387
2018-03-08 15:45:09,0.387,index180,sigmajin,0.387
2018-03-08 15:45:09,0.387,index181,sigmajin,0.387
2018-03-08 15:45:09,0.387,index182,sigmajin,0.387
2018-03-08 15:45:09,0.387,index183,sigmajin,0.387
2018-03-08 15:45:09,0.387,index184,sigmajin,0.387
2018-03-08 15:45:09,0.387,index185,sigmajin,0.387
2018-03-08 15:45:09,0.387,index186,sigmajin,0.387


In [83]:
rGraphData.tail()

,deposit,from,to,withdraw
2017-03-13 16:59:42,0.935,newbie6,originate,0.935
2017-03-13 16:58:45,0.898,newbie26,originate,0.898
2017-03-13 16:58:09,1.272,newbie25,originate,1.272
2017-03-13 16:57:36,1.670,newbie24,originate,1.670
2017-03-13 16:56:57,1.670,newbie23,originate,1.670


At first glance, I thought I had absolutely blown something in the translation. All of the from accounts appeared to be the same with the same timestamp – but then I looked closer. A lot closer. All of these transactions occurred during the same second, it's absolutely true. But they are being issued from a series of accounts with integer increasing names to a single withdrawal target.

Is this what a bot farm paying off looks like?

I have no idea what this is. But whatever it is, it's synchronized tightly enough so that all of these things are being issued at the same moment.

Looking at the other end of the record, the oldest things look pretty standard. Though, again, we see a situation in which a series of integer-named accounts are all withdrawing to the same account.

At least they're not all hitting the blockchain at once.

Again, what's going on here? I have no idea. Maybe @sigmajin and @originate would like to let us know?

Since we know that there are different numbers of from and to accounts, or at least that's the assumption we're going on for the moment, maybe we should figure out exactly how big the disparity between those two groups is.

That's pretty easy if we look at the data simply grouped by each of them separately.

In [84]:
rGraphData.groupby(['from', 'to']).sum()

deposit     withdraw
from             to                                    
a-ok             nrg                51.622       51.622
a00              nrg                40.528       40.528
a1r              nrg                40.622       40.622
a2v6aaz0tf       sigmajin           15.304       15.304
a4elentano       dart               40.231       40.231
a55              nrg                40.518       40.518
aa1wp9d4dr3      godheadpyqmh        0.143        0.143
                 smurarooseat       15.226       15.226
aa2cnromail      godheadpyqmh        0.143        0.143
                 sepoughtoate       14.788       14.788
aadsteemed       joeyd            2377.199     2377.199
aannabellaa      gidkit             13.588       13.588
aas0qtf70om      godheadpyqmh        0.143        0.143
                 smurarooseat       14.785       14.785
aaxdcs4vhlz      pheighezlesh       15.223       15.223
                 smurarooseat        0.143        0.143
aazwkxcrac       theytewhenu6        5.206        5.206
abcd             berkah            419.585      419.585
abcgomailw       semetelisloo        0.738        0.738
abdul            alpha           18153.027    18153.027
abdulham         dart               35.139       35.139
abelaper         gidkit             13.889       13.889
abhishe          sigmajin           15.128       15.128
abid             berkah            211.156      211.156
abigali          dart               13.686       13.686
abigayle         gidkit             13.577       13.577
abit             alittle        967910.288   967910.288
                 alot          1605035.164  1605035.164
abitminer1       alittle             8.445        8.445
abitminer2       alittle             2.302        2.302
...                                    ...          ...
zulule           gidkit              5.198        5.198
zulusardorilak   sigmajin           14.905       14.905
zulutaxe         gidkit              5.206        5.206
zuluzragoreketh  sigmajin           15.258       15.258
zuluzshura       gidkit              5.209        5.209
zulya            gidkit             14.258       14.258
zumie            dart               14.841       14.841
zumzum           gidkit             14.183       14.183
zunris           gidkit              5.220        5.220
zuppavgfjndd     phedesmkaquo       14.781       14.781
                 shotesltudud        0.143        0.143
zuran            gidkit              5.215        5.215
zurashaliwyn     dart                4.843        4.843
zurawsksckbw     godheadpyqmh       14.781       14.781
                 sepoughtoate        0.143        0.143
zurnmoralen      sigmajin           15.156       15.156
zushoshaktizil   dart               15.496       15.496
zusidaballadotus sigmajin           15.216       15.216
zvanr7k700       sigmajin           15.331       15.331
zwark            sepoughtoate       15.263       15.263
                 shotesltudud        0.143        0.143
zxc              dart               35.204       35.204
zxcvvb           gidkit              5.109        5.109
zyhzbvegyjm      phedesmkaquo        0.143        0.143
                 pheighezlesh       14.790       14.790
zyl              onthax            209.354      209.354
zynnnymehetele   dart               35.217       35.217
zzz              www             35819.052    35819.052
zzz3ya           dart               35.126       35.126
zzzya            dart               35.100       35.100

[23695 rows x 2 columns]

Well that now we know how that works out. Nearly 24,000 source accounts are going to just under 200 recipient accounts on a regular basis that nearly 1/3 of the traffic of this particular kind of operation on the blockchain, day in, day out, for the entirety of the last year.

I'm not going to say that such things look sketchy, but they look sketchy as Hell.

In [85]:
toRG = rGraphData.groupby(['to', 'from'])

toRG.sum()

deposit    withdraw
to           from                                
aaron        jesta         211542.903  211542.903
             powerbot-1      4255.104    4255.104
             powerbot-2      8486.685    8486.685
             powerbot-3      6477.957    6477.957
             powerbot-4      7806.740    7806.740
adiel        milutin2         113.785     113.785
             milutin2016       39.218      39.218
             ziona             30.402      30.402
             ziva              30.428      30.428
ahmeddimassi asmaasema          2.611       2.611
             steem-bot          0.735       0.735
aizensou     aizen            167.370     167.370
             aizen01          410.540     410.540
             aizen02          269.745     269.745
             aizen03          170.616     170.616
             aizen04           86.490      86.490
             aizen05           52.453      52.453
             aizen06          293.212     293.212
             aizen07          206.871     206.871
             aizen08          137.181     137.181
             aizen09           69.510      69.510
             aizen10           41.642      41.642
             aizen11           90.451      90.451
             aizen12          130.908     130.908
             aizen13           70.447      70.447
             aizen14          108.796     108.796
             aizen15           55.287      55.287
             aizen16          279.449     279.449
             aizen17          142.769     142.769
             aizen18           80.096      80.096
...                               ...         ...
yme          yme-00           235.788     235.788
             yme-01           182.137     182.137
             yme-02           152.129     152.129
             yme-03           142.312     142.312
             yme-04           138.489     138.489
             yme-05           128.946     128.946
             yme-06           126.841     126.841
             yme-07           119.951     119.951
             yme-08           114.017     114.017
             yme-09           108.285     108.285
             yme-10           102.737     102.737
             yme-11            98.896      98.896
             yme-12            94.122      94.122
             yme-13            88.764      88.764
             yme-14            84.359      84.359
             yme-15            78.824      78.824
             yme-16            74.234      74.234
             yme-17            68.887      68.887
             yme-18            64.695      64.695
             yme-19            60.876      60.876
             yme-20            57.252      57.252
             yme-21            53.442      53.442
             yme-22            51.343      51.343
             yme-23            48.875      48.875
             yme-24            45.829      45.829
zear         bellyrub        3454.115    3454.115
             bellyrubbank      66.964      66.964
             itzzia            57.687      57.687
             psych101          33.549      33.549
             zeartul         4690.089    4690.089

[23695 rows x 2 columns]

And there's our activity, all laid out neatly and effectively with all of the accounts which are pumping up into others completely revealed.

I find it interesting that, thinking back on the recent @bellyrub insanity, we can see what accounts that @zear has been involved with – at least as regards whatever this particular vesting operation represents.

All right, let's get on making that relationship graph. I'm more curious than ever to see what it looks like.

In [86]:
toNodes = set()
fromNodes = set()

for e in list(toRG.groups.keys()):
    (t, f) = e
    toNodes.add(t)
    fromNodes.add(f)    

In [87]:
len(toNodes), len(fromNodes)

(187, 21507)

In [88]:
for n in toNodes:
    dot.node(n)

In [89]:
dot.attr('node', fillcolor='lightgreen', shape='oval')

In [90]:
for n in fromNodes:
    dot.node(n)

Let's look at making the connecting edges a little more interesting. We'll label them with the actual sum of transfers that have gone along that road.

In [91]:
sumToRG = toRG.sum()

In [92]:
dot.attr('edge', fontcolor='red')

In [93]:
for e in sumToRG['deposit'].iteritems():
    (f, t), v = e 
    dot.edge(t, f, headlabel=str(v))

In [94]:
%time dot.render('02')

Wall time: 5min 2s


'02.svg'